In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pip install shap==0.23.0


In [ ]:
pip install workalendar

In [ ]:
import numpy as np
import pandas as pd

import shap

import lightgbm as lgbm

import matplotlib.pyplot as plt

from workalendar.asia import SouthKorea

from sklearn.impute import SimpleImputer # 결측치 처리 
from sklearn.preprocessing import LabelEncoder # 라벨 인코더
from sklearn.decomposition import TruncatedSVD # 특이값 분해
from sklearn.preprocessing import StandardScaler # 표준화
from sklearn.preprocessing import PolynomialFeatures # 다항회귀

In [ ]:
# Loading data.

data = pd.read_csv('/content/drive/MyDrive/dacon/funda_train.csv')

data['date'] = data['transacted_date'] + ' ' + data['transacted_time']
data['date'] = pd.to_datetime(data['date'])

data.head()

In [ ]:
data[data.amount < 0]

# 2. 데이터 전처리
Data Cleansing & Pre-Processing

In [ ]:
# Removing negative values. # 음수값 제거하기

def remove_negative(data):
    data_pos = data[data.amount > 0]
    data_neg = data[data.amount < 0]
    drop_ind = set()
    
    for neg in data_neg.itertuples():
        amount = abs(neg.amount) # 절대값
        
        row = data_pos[data_pos.store_id == neg.store_id] # 스토어 아이디
        row = row[row.card_id == neg.card_id] # 카드아이디
        row = row[row.card_company == neg.card_company] # 회사 같은지
        row = row[row.amount >= amount] # 
        row = row[row.date <= neg.date] # 음수가 더 많은 것?
        row = row[~row.index.isin(drop_ind)] # 중복제거하는거 같은데 
        
        if len(row[row.amount == amount]) > 0: # 값이 같은지
            row = row[row.amount == amount]
            matched_row = row[row.date == max(row.date)]
            drop_ind.update(matched_row.index) # matched_row의 index를 가져와서 drop_ind를 설정하고
        elif len(row[row.amount > 0]):
            matched_row = row[row.date == max(row.date)]
            data_pos.loc[matched_row.index, 'amount'] += neg.amount
            
    data_pos.drop(drop_ind, axis = 0, inplace = True) # 제거한다. 
    print('Count - with negative: {}'.format(len(data)))
    print('Count - without negative: {}'.format(len(data_pos)))

    return data_pos
    
data = remove_negative(data)

In [ ]:
# Removing unnecessary columns. # 안쓰는 컬럼 지우고 

drop_cols = ['installment_term', 'transacted_date', 'transacted_time']
data.drop(drop_cols, axis = 1, inplace = True)

data.head()

In [ ]:
# Business-related features expansion. (비즈니스 관련 기능 확장)

# I grouped different bussiness in smaller categories and organized them into a separate .CSV ( 여러 비즈니스를 더 작은 범주로 그룹화하고 별도의 .CSV로 구성했습니다.)

biz_info = pd.read_csv('../data/business_info.csv')
data = data.merge(biz_info, on = 'type_of_business', how = 'left')

# Numerating strings as categories.

encoder = LabelEncoder()


# I grouped different bussiness in smaller categories and organized them into a separate .CSV

biz_info = pd.read_csv('../data/business_info.csv')
data = data.merge(biz_info, on = 'type_of_business', how = 'left')

# Numerating strings as categories.

encoder = LabelEncoder() # 라벨인코더 하고 
text_cols = data.dtypes.pipe(lambda x: x[x == 'object']).index.tolist()
data[text_cols] = data[text_cols].apply(lambda x: encoder.fit_transform(x.tolist()))

data.head()

In [ ]:
# Date-related features extraction and preprocessing. #날짜관련 기능 전처리 

cal = SouthKorea()

data['weekday'] = data['date'].dt.weekday # 주차
data['holiday'] = data['date'].apply(lambda x: cal.is_holiday(x)).astype(int) # 휴일
data['workday'] = data['date'].apply(lambda x: cal.is_working_day(x)).astype(int) # workday

data.set_index('date', drop = False, inplace = True) # date를 인덱스로 사용
data.sort_index(inplace = True)

data.head()

In [ ]:
# Active stores. #활동하고 있는 매장 구하기 

active_stores = data.last('3M')['store_id'].unique() # 스토어의 유니크 키를 가져오고
data['is_active'] = data['store_id'].isin(active_stores).astype(int) #지금 활동하고 있는 스토어를 가져온다.

# Global aggregates by store.

transforms = ['sum', 'mean', 'median', 'size', 'std', 'min', 'max']
group = data.groupby('store_id')

for t in transforms:  # 각각 지점의 합 , 평균, 중앙, 크기 , 표준편차, 최소 , 최대값 구하기
    data[t] = group['amount'].transform(t)

data.head()

In [ ]:
# One-hot encoding. # 더미화 하기 

dummy_cols = ['card_company', 'weekday'] # 이 두개만 따로

for col in dummy_cols:
    data = pd.concat([data, pd.get_dummies(data[col], prefix = col)], axis=1)

data.drop(dummy_cols, axis = 1, inplace = True)

data.head()

# 3. 탐색적 자료분석
Exploratory Data Analysis

In [ ]:
# Adding moving averages and last-N stats for 3 months groups.
# 3 개월 그룹에 대한 이동 평균 및 마지막 N 통계 추가.

data['group_id'] = data.groupby(pd.Grouper(level = 0, freq = '1QS-DEC')).ngroup() # 날짜를 월별로 그룹화 한 것을 변수로 저장

days = [1, 7, 14, 28, 46]
sid_list = data['store_id'].unique() # 유니크한 스토어 아이디
gid_list = data['group_id'].unique() # 유니크한 그룹 아이디

sales = data.groupby(['store_id', 'group_id', pd.Grouper(level = 0, freq = '1D')])[['amount']].sum() # 이 두가지 그룹한 것을 토대로 합계를 구해 판매액을 구한다.
features = []

for sid in sid_list:
    for gid in gid_list:
        try:
            store = sales.xs(sid, level = 0).xs(gid, level = 0)
        except Exception as error:
            continue
            
        dictionary = {'store_id': sid, 'group_id': gid}
        
        # Calculating decay rates. 붕괴율 계산? 단위시간당 분해되는 회계수 
        
        for d in days:
            amount = store['amount'].resample('{}D'.format(d)).sum(min_count = 1).dropna()
            date_dif = amount.reset_index()['date'].diff(-1).dt.days.abs()
            
            amount_dif = (amount.shift(-1) / amount).reset_index(drop = True)
            dictionary['decay{}_mean'.format(d)] = amount_dif.mean()
            dictionary['decay{}_mean~time'.format(d)] = (amount_dif / date_dif).mean()
            
            amount_dif = np.log1p(amount_dif)
            dictionary['decay{}_log_mean'.format(d)] = (amount_dif / date_dif).mean()
            dictionary['decay{}_log_mean~time'.format(d)] = (amount_dif / date_dif).mean()
            
        for d in days: 
            store['ma{}'.format(d)] = store['amount'].rolling('{}D'.format(d)).mean()
            
        for d1 in days:
            l = store.last('{}D'.format(d1))
            
            dictionary['last{}_sum'.format(d1)] = l.amount.sum()
            dictionary['last{}_mean'.format(d1)] = l.amount.mean()
            dictionary['last{}_median'.format(d1)] = l.amount.median()
            
            for d2 in days:
                dictionary['ma{}_last{}_mean'.format(d2, d1)] = l['ma{}'.format(d2)].mean()
                
        features.append(dictionary)
            
features = pd.DataFrame(features)

features.head()

In [ ]:
# Appending new features. (새로운 기능)

ndata = data.merge(features, on = ['group_id', 'store_id'], how = 'left')
ndata.set_index('date', drop = False, inplace = True)
ndata.sort_index(inplace = True)

# Aggregating data by 3 months duration.(3개월 기간으로 데이터 집계)

aggregations = {
    'amount': {
        'sales_current': 'sum',
        'sales_mean': 'mean',
        'sales_median': 'median',
        'sales_count': 'count',
        'sales_std': 'std',
        'sales_mad': 'mad',
        'sales_min': 'min',
        'sales_max': 'max'
    },
    'holiday': {
        'holidays_%': 'mean',
        'holidays_count': 'sum'
    },
    'workday': {
        'workdays_%': 'mean',
        'workdays_count': 'sum'
    },
    'card_id': {
        'customers_unique': 'nunique',
    },
    'date': {
        'year_min': lambda x: min(x).year,
        'year_max': lambda x: max(x).year,
        'doty_min': lambda x: min(x).dayofyear,
        'doty_max': lambda x: max(x).dayofyear,
        'time_span': lambda x: (max(x) - min(x)).days + 1
    }
}

dummy_cols_agg = {col: {col: 'mean'} for col in ndata.columns if col.startswith(tuple(dummy_cols))}
aggregations.update(dummy_cols_agg)

col_regex = '|'.join(transforms)
first_cols = ['region', 'type_of_business', 'business_area', 'kind_of_service', 'is_active']
first_cols.extend(ndata.columns[ndata.columns.str.contains(col_regex, regex = True)])
first_cols_agg = {col: {col: 'first'} for col in first_cols}
aggregations.update(first_cols_agg)

ndata = ndata.groupby(['store_id', pd.Grouper(level = 0, freq = '1QS-DEC')])
ndata = ndata.agg(aggregations)
ndata.columns = ndata.columns.droplevel(0)

# Sales and moving averages lags.

gdata = ndata.groupby(level = 0)
lag_cols = set(first_cols) - set(transforms) - set(text_cols)

for i in range(1, 9):
    ndata['sales_diff_{}'.format(i)] = gdata['sales_current'].diff(i)
    ndata['sales_last_{}'.format(i)] = gdata['sales_current'].shift(i)
    ndata['sales_pctd_{}'.format(i)] = gdata['sales_current'].pct_change(i)
    
    ndata['sales_diff_last{}'.format(i)] = gdata['sales_current'].diff().shift(i)
    ndata['sales_pctd_last{}'.format(i)] = gdata['sales_current'].pct_change().shift(i)
    
    ndata['sales_last_{}_current_%'.format(i)] = ndata['sales_last_{}'.format(i)] / ndata['sales_current']
    ndata['sales_current_last_{}_%'.format(i)] = ndata['sales_current'] / ndata['sales_last_{}'.format(i)]
    ndata['sales_dcay_{}'.format(i)] = ndata['sales_current'] / ndata['sales_last_{}'.format(i)]
    ndata['sales_dcay{}_log'.format(i)] = np.log1p(ndata['sales_dcay_{}'.format(i)])
    
    for col in lag_cols:
        ndata['{}_lag_{}'.format(col, i)] = gdata[col].shift(i)
    
# Additional features and feature interactions.(추가기능 및 기능 상호 작용)

ndata['sales_mean_%'] = ndata['sales_mean'] / ndata['mean']
ndata['sales_range'] = ndata['sales_max'] - ndata['sales_min']
ndata['customers_unique_%'] = ndata['customers_unique'] / ndata['sales_count']
ndata['sales_diff_1_mean'] = ndata.groupby(level = 0)['sales_diff_1'].transform('mean')
ndata['sales_pctd_1_mean'] = ndata.groupby(level = 0)['sales_pctd_1'].transform('mean')
ndata['sales_current_last_1_%_mean'] = ndata.groupby(level = 0)['sales_current_last_1_%'].transform('mean')

for t in transforms: 
    ndata['rank_{}'.format(t)] = ndata[t].rank(method = 'dense', ascending = False)   
    
for d in days:
    ndata['last{}_sum_current_%'.format(d)] = ndata['last{}_sum'.format(d)] / ndata['sales_current']    
    
ndata['range'] = ndata['max'] - ndata['min']
ndata['rank_range'] = ndata['range'].rank(method = 'dense', ascending = False)
ndata['sales_current_rank_sum_%'] = ndata['sales_current'] / ndata['rank_sum']

for i in range(2, 9):
    ndata['sales_diff_{}_diff_1_mean_%'.format(i)] = ndata['sales_diff_{}'.format(i)] / ndata['sales_diff_1_mean']
    ndata['sales_pctd_{}_pctd_1_mean_%'.format(i)] = ndata['sales_pctd_{}'.format(i)] / ndata['sales_pctd_1_mean']
    ndata['sales_pctd_{}_diff_1_mean_%'.format(i)] = ndata['sales_pctd_{}'.format(i)] / ndata['sales_diff_1_mean']
    ndata['sales_diff_{}_pctd_1_mean_%'.format(i)] = ndata['sales_diff_{}'.format(i)] / ndata['sales_pctd_1_mean']

    
# Post-processing.
    
ndata.drop(['sum', 'size'], axis = 1, inplace = True)
ndata.reset_index(inplace = True)    
ndata.head()

In [ ]:
# Adding truncated SVD features. svd 알고리즘 돌리기 

imp = SimpleImputer(strategy = "constant", fill_value = 0) # 
tr_svd = TruncatedSVD(n_components = 20)
ex_cols = ['date', 'store_id', 'sales_current', 'sales_current_last_1_%', 'is_active',
           'region', 'type_of_business', 'business_area', 'kind_of_service']

x = ndata.loc[:, ~ndata.columns.isin(ex_cols)]
scaled_x = StandardScaler().fit_transform(x.values) # 표준화
imputed_x = imp.fit_transform(scaled_x) # 빈 값 대체하기 
truncated_x = tr_svd.fit_transform(imputed_x) # svd 돌리기

scaled_x = pd.DataFrame(scaled_x, columns = x.columns) # scaled_x 에 대한 데이터 프레임 만들고
truncated_x = pd.DataFrame(truncated_x) # truncated_x 에 대한 데이터 프레임만든다.

tdata = pd.concat([ndata[ex_cols], scaled_x, truncated_x], axis = 1) # 그리고 합친다.
tdata.head()

#4. 변수 선택 및 모델 구축
Feature Engineering & Initial Modeling

In [ ]:
# Preparing training and validation datasets. (훈련 및 검증 데이터세트 구하기)

def split_data(data, tr_multiplier = 1, is_expanding = False, verbose = True):
    n = len(data.date.unique()) - (tr_multiplier + 1)
    group = list(data.groupby('date')) # date로 그룹화  한 것을 리스트로 가져온다. 
    group = [g[-1] for g in group] # 그룹하 한것을 하나씩 저장한다.
    splits = []
    
    for i in range(0, n):
        ceil = i + tr_multiplier
        floor = 0 if is_expanding else i

        train = pd.concat(group[floor : ceil])
        train = train.sort_values('date').groupby('store_id').tail(1)
        valid = group[ceil: ceil + 1][0]
        y_val = []
        
        if verbose:
            print('Fold {}: TRAIN - {} to {}'.format(i, train.date.min(), valid.date.min()))

        for j in range(0, 2):
            y = group[ceil + j : ceil + j + 1][0][['store_id', 'sales_current_last_1_%']]
            y['sales_current_last_1_%'] = np.log1p(np.log1p(y['sales_current_last_1_%']))
            y.columns = ['store_id', 'y']
            y_val.append(y)
            
        train = train.merge(y_val[0], on = 'store_id', how = 'inner')
        valid = valid.merge(y_val[-1], on = 'store_id', how = 'inner')

        train = train.drop('date', axis = 1).reset_index(drop = True)
        valid = valid.drop('date', axis = 1).reset_index(drop = True)
        
        splits.append((train, valid))

    return splits


splits = split_data(tdata, 1, True)
splits[0][0].head()

In [ ]:
# Fitting model and cross validation. (피팅 모델 및 교차검증하기.)

metric = 'l1'
cat_features = ['region', 'type_of_business', 'business_area', 'kind_of_service']

def train_lgbm(splits, metric = 'l1', categories = '',
               max_depth = 6, num_leaves = 48, feature_fraction = 0.8,
               num_iterations = 3000, early_stopping = 100):
    models = []
    params = {
        'learning_rate': 0.01,
        'boosting': 'gbdt',
        'objective': 'regression_l1',
        'num_leaves': num_leaves,
        'max_bin': 255,
        'max_depth': max_depth,
        'metric': metric,
        'num_iterations': num_iterations,
        'early_stopping': early_stopping,
        'cat_smooth': 10,
        'feature_fraction': feature_fraction
    }

    for (t, v) in splits:
        d_train = lgbm.Dataset(t.drop('y', axis = 1), label = t['y'])
        d_valid = lgbm.Dataset(v.drop('y', axis = 1), label = v['y'], reference = d_train)

        evals_result = {}
        clf = lgbm.train(params, d_train, valid_sets = [d_train, d_valid], 
                         evals_result = evals_result, verbose_eval = 1500,
                         categorical_feature = categories)

        model = {
            'score': list(clf.best_score.values()),
            'evals': evals_result,
            'model': clf,
            'best': clf.best_iteration
        }

        models.append((model, t, v))
        
    return models

                         
models = train_lgbm(splits, metric)

In [ ]:
scores = [m['score'] for (m, _, _) in models]
scores = [{'train': score[0][metric], 'valid': score[-1][metric]} for score in scores]
scores = pd.DataFrame(scores)
scores.describe()

# 5. 모델 학습 및 검증
Model Tuning & Evaluation

In [ ]:
for (m, _, _) in models:
    lgbm.plot_importance(m['model'], importance_type = 'gain', max_num_features = 15, figsize = (15, 6))
    plt.show()

In [ ]:
for (m, _, _) in models:
    lgbm.plot_metric(m['evals'], metric = metric, figsize = (15, 6))
    plt.show()

In [ ]:
# Making and plotting predictions for stores, which are common across all folds. (# 매장에 대한 예측을 만들고 플로팅합니다. 이는 모든 접기에 공통입니다.)

sid_list = set()

for (_, t, v) in models:    
    ids = set(t.store_id) & set(v.store_id)
    sid_list = sid_list & ids if bool(sid_list) else ids
    
for (m, _, v) in models:    
    d_plot = v[v['store_id'].isin(sid_list)]
    d_plot['y_pred'] = m['model'].predict(d_plot, num_iteration = m['best'])
    d_plot = np.expm1(np.expm1(d_plot.set_index('store_id')[['y', 'y_pred']]))
    d_plot['error'] = d_plot['y'] - d_plot['y_pred']
    d_plot = d_plot.groupby('store_id')['error'].mean()
    d_plot.sort_values(inplace = True)

    d_plot.iloc[::20].plot.bar(figsize = (15, 6), title = 'Prediction error')
    plt.show()

In [ ]:
# Exploring feature interactions with SHAP.

def plot_feature_interaction(data, interaction_values): 
    img = np.abs(interaction_values).sum(0)

    for i in range(img.shape[0]):
        img[i,i] = 0

    inds = np.argsort(-img.sum(0))[:30]
    img = img[inds,:][:,inds]

    plt.figure(figsize = (10, 10))
    plt.imshow(img)
    plt.yticks(range(img.shape[0]), data.columns[inds], rotation = 0, horizontalalignment = "right")
    plt.xticks(range(img.shape[0]), data.columns[inds], rotation = 50, horizontalalignment = "left")
    plt.gca().xaxis.tick_top()
    plt.show()

for (m, _, v) in models:
    valid = v.drop('y', axis = 1)
    explainer = shap.TreeExplainer(m['model'])
    shap_interaction_values = explainer.shap_interaction_values(valid)
    plot_feature_interaction(valid, shap_interaction_values)

In [ ]:
# Adding error-based features.

def add_error_features(data, models):
    errors = pd.DataFrame() 
    
    for (m, t, v) in models:    
        valid = v[['store_id', 'y']]
        train = t[['store_id', 'y']]

        valid['y_pred'] = m['model'].predict(v, num_iteration = m['best'])
        train['y_pred'] = m['model'].predict(t, num_iteration = m['best'])

        train = np.expm1(np.expm1(train.set_index('store_id')[['y', 'y_pred']])).reset_index()
        valid = np.expm1(np.expm1(valid.set_index('store_id')[['y', 'y_pred']])).reset_index()

        train['error'] = train['y'] - train['y_pred']
        valid['error'] = valid['y'] - valid['y_pred']

        merged = train.merge(valid, on = 'store_id', how = 'outer')[['store_id', 'error_x', 'error_y']]
        errors = pd.concat([errors, merged], axis = 0)

    errors = errors.groupby('store_id')[['error_x', 'error_y']].mean().reset_index()
    errors['error_x_abs'], errors['error_y_abs'] = errors['error_x'].abs(), errors['error_y'].abs()
    errors['error_y_x_%'] = errors['error_y'] / errors['error_x']

    errors['error_abs_sum'] = errors['error_x_abs'] + errors['error_y_abs']
    errors['error_abs_mean'] = 0.5 * errors['error_abs_sum'] 
    errors['error_sum'] = errors['error_x'] + errors['error_y']
    errors['error_mean'] = 0.5 * errors['error_sum']

    quantiles = [0, .01, .05, .2, .5, .8, .95, .99, 1]

    error_cols = [col for col in errors.columns if col.startswith('error')]
    for col in error_cols:
        errors['{}_group'.format(col)] = pd.qcut(errors[col], quantiles, labels = False)
        errors['rank_{}'.format(col)] = errors[col].rank(method = 'dense')

    erdata = data.merge(errors, on = 'store_id', how = 'left')
    
    return erdata

    
erdata = add_error_features(tdata, models)
erdata.head()

In [ ]:
# We retrain the model with error-based features.

ersplits = split_data(erdata, 1, True, False)
ermodels = train_lgbm(ersplits, metric)

In [ ]:
scores = [m['score'] for (m, _, _) in ermodels]
scores = [{'train': score[0][metric], 'valid': score[-1][metric]} for score in scores]
scores = pd.DataFrame(scores)
scores.describe()

In [ ]:
# We remove all of the features, which are not giving enough gains.

important_features = set()

for (m, _, v) in ermodels:
    valid = v.drop('y', axis = 1)
    explainer = shap.TreeExplainer(m['model'])
    shap_values = explainer.shap_values(valid)
    shap_sum = np.abs(shap_values).mean(axis=0)

    importance_df = pd.DataFrame([valid.columns.tolist(), shap_sum.tolist()]).T
    importance_df.columns = ['feature', 'shap_importance']
    important_features |= set(importance_df[importance_df['shap_importance'] > 10 ** -3]['feature'])
      
len(important_features)

In [ ]:
idata = erdata[important_features | set(['date', 'store_id', 'sales_current', 'sales_current_last_1_%', 'is_active'])]
isplits = split_data(idata, 1, True, False)
imodels = train_lgbm(isplits, metric, max_depth = 5, num_leaves = 32, 
                     feature_fraction = 0.9, num_iterations = 2000, early_stopping = 50)


In [ ]:
# We reorganize features by adding features interactions for most imporant features and then
# (# 우리는 가장 중요한 기능에 대한 기능 상호 작용을 추가하여 기능을 재구성 한 다음)
# factorizing the matrix by tr-svd.
# tr-svd로 행렬 분해.


poly = PolynomialFeatures(2, include_bias = False, interaction_only = True)

z = idata.loc[:, ~idata.columns.isin(ex_cols)]
imputed_z = imp.fit_transform(z.values)
poly_z = poly.fit_transform(imputed_z)
truncated_z = tr_svd.fit_transform(poly_z)
truncated_z = pd.DataFrame(truncated_x)

zdata = pd.concat([idata, truncated_z], axis = 1)
zdata.head()

In [ ]:
zsplits = split_data(zdata, 1, True, False)
zmodels = train_lgbm(zsplits, metric, max_depth = 5, num_leaves = 32, 
                     feature_fraction = 0.9, num_iterations = 2000, early_stopping = 50)


# 6. 결과 및 결언
Conclusion & Discussion


In [ ]:
# Getting submission data.

def get_submission(sbm_data, models, average_out = False):    
    test = sbm_data.sort_values('date').groupby('store_id').tail(1).drop('date', axis = 1)
    test.sort_values('store_id', inplace = True)

    m = models[-1][0]
    prediction = m['model'].predict(test, num_iteration = m['best'])
    prediction = np.expm1(np.expm1(prediction))

    submission = test[['store_id', 'sales_current', 'is_active']]
    submission['amount'] = submission['sales_current'] * prediction
    submission.set_index('store_id', inplace = True)
    print(submission['amount'].mean())
    
    if average_out:
        submission['amount'] = 0.5 * submission['amount'] + 0.5 * submission['sales_current']
        print(submission['amount'].mean())
        
    submission.loc[submission['is_active'] == 0, 'amount'] = 0
    submission = submission[['amount']]
    print(submission['amount'].mean())
    
    return submission
    

submission = get_submission(zdata, zmodels, True)    
# submission.to_csv('../notes/submissions/submission.csv')

In [ ]:
# Final thoughts:
#
# This model due to the nature of preprocessing suffers from leaks, but I didn't have
# enough time to get through reengineering process.
# It would have probably been a good idea to keep information about installments
# and use it as an estimation for level of luxury for different shops.
# I didn't work with N/A values in my solution and just left them as a separate category,
# but playing around with using KNN to fill them might have given better results.
# The worst part about this model is that it uses post-processing by averaging out results
# of the last 3 months. 
# Another way to improve the model would have been to train many Lasso-regression models
# separately for each store and use their predictions as features for LGBM model.
# Working with lower granularity of data might have worked as well.
#
# Thank you!


# 마지막 생각들:
#
# 이 모델은 전처리의 특성상 누수가 있지만
# 리엔지니어링 과정을 거치기에 충분한 시간.
# 할부 정보를 보관하는 것이 좋은 생각이었을 것입니다.
# 다른 상점의 명품 수준에 대한 추정치로 사용합니다.
# 내 솔루션에서 N / A 값으로 작업하지 않고 별도의 범주로 남겨 두었습니다.
# 그러나 KNN을 사용하여이를 채우는 것이 더 나은 결과를 제공했을 수 있습니다.
# 이 모델의 가장 나쁜 점은 결과를 평균화하여 후 처리를 사용한다는 것입니다.
# 지난 3 개월 중 
# 모델을 개선하는 또 다른 방법은 많은 Lasso-regression 모델을 훈련시키는 것입니다.
# 각 상점에 대해 개별적으로 예측을 LGBM 모델의 기능으로 사용하십시오.
# 낮은 단위의 데이터로 작업하는 것도 효과가있을 수 있습니다.
#
# 감사합니다!

In [ ]:
데이터 전처리
1.카테고리 분류 다시함
2. 음수를 제거함 
3.안쓰는 컬럼제거
4.휴일,일하는일, weekday
5.실제로 운영하고 있는지에 대한 변수를 생성
6.store_id그룹 - 기초 통계

탐색적 자료분석
1. 스토어 아이디 + 그룹아이디(아마도 월별 ) 묶어서 그룹화


회귀를 했을떄 독립변수들이 여러개 있을때 

최적의 모델을 변수를 선택하는데 back 은 제일 적은거다. 

변수선택법을 

ㅇㅋㅇㅋ 



데이터 전처리
1.카테고리 분류 다시함
2. 음수를 제거함 
3.안쓰는 컬럼제거
4.휴일,일하는일, weekday
5.실제로 운영하고 있는지에 대한 변수를 생성
6.store_id그룹 - 기초 통계

탐색적 자료분석
1. 스토어 아이디 + 그룹아이디(아마도 월별 ) 묶어서 그룹화


회귀를 했을떄 독립변수들이 여러개 있을때 

최적의 모델을 변수를 선택하는데 back 은 제일 적은거다. 

변수선택법 




PolynomialFeatures

다항 회귀는 비선형 데이터를 학습하기 위한 선형 모델인데
변수를 뭐 거듭제곱하던지 해서 변수를 확장해서 훈련하는거래욥
그래서 우리 데이터가 비선형 시계열 자료라서 다항회귀한듯


비선형 효과를 회귀분석에 담기 위해 회귀모형을 확장하는 몇 가지 방법이 있다.
다항식 회귀
스플라인 회귀
일반화가법모델(GAM)
시계열데이터인 경우 AR, MA, ARIMA 등
머신러닝 모델

좋았다 집단지성 굳 

PolynomialFeatures
